# Data Overview

* 🏭 V1, V2 (same company)
    - cloud_v00001.csv & cloud_v00002.csv ()
        + v1: 250322 - 010323 - 342 day (338 day in dataset)
        + v2: 250322 - 010323 - 342 day (338 day in dataset)
        + y6: 250322 - 010323 - 342 day (338 day in dataset)
        + y7: 250322 - 010323 - 342 day (339 day in dataset)
        + execution_date
        + target_date
        + cloud(%)
    - solar_v00001.csv & solar_v00002.csv
        + v1: 250322 - 010323 - 342 day (330 day in dataset)
        + v2: 250322 - 010323 - 342 day (327 day in dataset)
        + y6: 250322 - 010323 - 342 day (331 day in dataset)
        + y7: 250322 - 010323 - 342 day (329 day in dataset)
        + execution_date
        + target_date
        + solar(W/m2)
    - weather_forecast_v00001.csv & weather_forecast_v00002.csv
        + v1: 250322 - 010323 - 342 day (330 day in dataset)
        + v2: 250322 - 010323 - 342 day (327 day in dataset)
        + y6: 250322 - 010323 - 342 day (331 day in dataset)
        + y7: 250322 - 010323 - 342 day (328 day in dataset)
        + execution_data
        + target_date
        + telop_code
        + telop_name
    - realne_report_solar_30_v00001_20220301_20230228.xlsx & realne_report_solar_30_v00002_20220301_20230228.xlsx
        + completed data (365 day)
        + 時刻 (time)
        + 30001（Wh）
        + 30011（Wh）
        + 30021（Wh）
        + 30031（Wh）
    - realne_report_surplus30p_v00001_20220301_20221231.xlsx & realne_report_surplus30p_v00002_20220301_20221231.xlsx
        + completed data (365 day)
        + 時刻 (time)
        + 30101:電力量（Wh）
        + 30101:回生電力量（Wh）
    - realne_report_surplus30p_v00001_20230101_20230228.xlsx & realne_report_surplus30p_v00002_20230101_20230228.xlsx
* 🏭 Y6
* 🏭 Y7

=> From 25/03/2022 to 28/02/2023 (341 day)

5 file
2 output
    1/3/2022 -> 28/2/2023
3 input
    25/3/2022 -> 1/3/2023


# Import Lib

In [12]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import itertools
import functools
import operator
import json

# from datetime import datetime, timedelta

# Helper

In [54]:
def get_missingdays(days:np.ndarray):
    days = np.unique(days)
    days = np.asarray([date(int(i[:4]), int(i[5:7]), int(i[8:10])) for i in days])
    # print(days)
    res = []
    for i, v in enumerate(days):
        if (i==0): continue
        delta = (days[i] - days[i-1]).days
        if delta > 1:
            for _i in range(0, delta-1):
                next_day = days[i-1 + _i] + timedelta(days=1)
                res = res + [next_day]
                
    # print(res)
    res_str = [i.strftime('%Y-%m-%d') for i in res]
    # date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    return res_str

def gen_dates(date_begin, date_end, x48 = True):
    y1,m1,d1 = [int(i) for i in date_begin.split("-")]
    y2,m2,d2 = [int(i) for i in date_end.split("-")]
    date_begin_dt = date(y1, m1, d1)
    date_end_dt =   date(y2, m2, d2)

    date_list_dt = np.arange(date_begin_dt, date_end_dt + timedelta(days=1), timedelta(days=1)).astype(date)
    date_list = [i.strftime('%Y-%m-%d') for i in date_list_dt]
    # print(date_list)
    res =  [([i] * 48) for i in date_list]
    # print (res)
    return functools.reduce(lambda x, y: x + y, res) if (x48) else date_list



    
# gen_dates("2023-01-01", "2023-02-02")

# Class design

In [55]:
Time_points = ['00:00', '00:30', '01:00', '01:30', '02:00', '02:30', '03:00', '03:30', '04:00', '04:30', '05:00', '05:30', '06:00', '06:30', '07:00', '07:30', '08:00', '08:30', '09:00', '09:30', '10:00', '10:30', '11:00', '11:30', '12:00', '12:30', '13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '16:30', '17:00', '17:30', '18:00', '18:30', '19:00', '19:30', '20:00', '20:30', '21:00', '21:30', '22:00', '22:30', '23:00', '23:30']
Dates_1to24 = gen_dates("2022-03-01", "2022-03-24")

date_list_x1 = gen_dates("2022-03-01", "2023-03-01", x48=False)

In [89]:
class JoinData:
    def __init__(self, loc = "v1", folder = "./2023_devday_data_version1/"):
        self.folder = folder
        self.loc = loc
        self.data_join = "Not joined"
        
        self.RAW_cloud = pd.read_csv(folder + loc + "/cloud_" + loc[0] +  "0000" + loc[1] + ".csv")
        self.RAW_solar = pd.read_csv(folder + loc + "/solar_" + loc[0] +  "0000" + loc[1] + ".csv")
        self.RAW_weather = pd.read_csv(folder + loc + "/weather_forecast_" + loc[0] +  "0000" + loc[1] + ".csv")
        
        self.f_real_solar = folder + loc + "/realne_report_solar_30_" + loc[0] +  "0000" + loc[1] + "_20220301_20230228.xlsx"
        self.RAW_report_solar = pd.ExcelFile(self.f_real_solar)
        if (loc in ["v1", "v2"]):
            self.RAW_report_surplus_2022 = pd.ExcelFile(folder + loc + "/realne_report_surplus30p_" + loc[0] +  "0000" + loc[1] + "_20220301_20221231.xlsx")
            self.RAW_report_surplus_2023 = pd.ExcelFile(folder + loc + "/realne_report_surplus30p_" + loc[0] +  "0000" + loc[1] + "_20230101_20230228.xlsx")
        else:
            self.RAW_report_surplus = pd.ExcelFile(folder + loc + "/realne_report_surplus30p_" + loc[0] +  "0000" + loc[1] + "_20220301_20230228.xlsx")
            
        # real_surplus
        
        ft_cloud = self.get_featurelist(self.RAW_cloud)
        ft_solar = self.get_featurelist(self.RAW_solar)
        ft_telop = self.get_featurelist(self.RAW_weather)
        
        self.join_feature(ft_cloud = ft_cloud, ft_solar=ft_solar, ft_telop = ft_telop)
        self.join_real()
        
        
    def get_featurelist(self, raw_: pd.DataFrame):
        ft = raw_.copy()
        ft = ft.sort_values('target_date', ascending=True)
        datesraw = np.asarray([x [:10] for x in ft["target_date"]])
        
        dates_unique = np.unique(datesraw)
        datesmiss = get_missingdays(dates_unique)
        
        table_add = pd.DataFrame(columns=raw_.columns)
        datesaddX48 = [ i + "T" for i in Dates_1to24] + functools.reduce(lambda x, y: x + y, [([i+"T"] * 48) for i in datesmiss])
        targettime_add = list(map(operator.add, datesaddX48, Time_points * 366))
        table_add['target_date'] = pd.Series(targettime_add)
        
        if ('cloud(%)' in  raw_.columns):
            self.cloud_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)
            return self.cloud_clean['cloud(%)'].tolist()
        
        if ('solar(W/m2)' in  raw_.columns):
            self.solar_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)
            return self.solar_clean['solar(W/m2)'].tolist()
        
        if ('telop_code' in  raw_.columns):
            self.telopcode_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)
            return self.telopcode_clean['telop_code'].tolist()
        
        
    def join_feature(self, ft_cloud = "", ft_solar = "", ft_telop = ""):
        self.data_join = pd.read_excel("join.xlsx", sheet_name = "template_" + self.loc)
        self.data_join['date'] = pd.Series(gen_dates("2022-03-01", "2023-03-01"))
        self.data_join['time'] = pd.Series(Time_points*366)
        self.data_join['datetime'] = pd.Series(list(map(operator.add, self.data_join['date'],[' ' + i for i in self.data_join['time']])))
        self.data_join['year'] = [i[:4] for i in self.data_join['date']]
        self.data_join['month'] = [i[5:7] for i in self.data_join['date']]
        self.data_join['day'] = [i[8:10] for i in self.data_join['date']]
        
        if ft_cloud: 
            self.data_join['cloud'] = pd.Series(ft_cloud)
        
        if ft_solar: 
            self.data_join['solar'] = pd.Series(ft_solar)
        if ft_telop:
            self.data_join['telop_code'] = pd.Series(ft_telop)
            
            ft_telop_clean = list(map(lambda x: 0 if np.isnan(x) else x, ft_telop))
            ft_telop_eng = []
            with open('telop.json') as json_telop:
                telop_dict = json.load(json_telop)
            ft_telop_eng = [telop_dict[str(int(i))] for i in ft_telop_clean]
            self.data_join['telop_eng'] = pd.Series(ft_telop_eng)

    def join_real(self):
        for index, row in self.data_join.iterrows():
            # print(row['date'])
            # if (index == 60): break
            
            if row['date'] == '2023-03-01': break
            curr_timep = index % 48
            if (curr_timep == 0):
                solar_currdate = pd.read_excel(self.RAW_report_solar, date_list_x1[int(index / 48)], skiprows = 6, header=None)
                if (self.loc[0] == 'y'):
                    surplus_currdate = pd.read_excel(self.RAW_report_surplus, date_list_x1[int(index / 48)], skiprows = 6, header=None)
                else:
                    surplus_currdate = pd.read_excel(self.RAW_report_surplus_2022, date_list_x1[int(index / 48)], skiprows = 6, header=None) \
                        if row['year'] == '2022' else pd.read_excel(self.RAW_report_surplus_2023, date_list_x1[int(index / 48)], skiprows = 6, header=None)
                    
            
            if (self.loc == "v1"):
                # print('ok')
                self.data_join.at[index, 'real_s30001'] = solar_currdate.iloc[curr_timep, 1]
                self.data_join.at[index, 'real_s30011'] = solar_currdate.iloc[curr_timep, 2]
                self.data_join.at[index, 'real_s30021'] = solar_currdate.iloc[curr_timep, 3]
                self.data_join.at[index, 'real_s30031'] = solar_currdate.iloc[curr_timep, 4]
                self.data_join.at[index, 'real_ssum'] = self.data_join.at[index, 'real_s30001'] + self.data_join.at[index, 'real_s30011'] + self.data_join.at[index, 'real_s30021'] + self.data_join.at[index, 'real_s30031']
                
                # row['real_s30001'] = solar_currdate.iloc[curr_timep, 1]
                # row['real_s30011'] = solar_currdate.iloc[curr_timep, 2]
                # row['real_s30021'] = solar_currdate.iloc[curr_timep, 3]
                # row['real_s30031'] = solar_currdate.iloc[curr_timep, 4]
        
            # print(solar_currdate.iloc[curr_timep, 1])
            # print(row['real_s30001'] )
            
            # Surplus
            self.data_join.at[index, 'real_demand'] = surplus_currdate.iloc[curr_timep, 1]
            self.data_join.at[index, 'real_surplus'] = surplus_currdate.iloc[curr_timep, 2]
            
    def to_xlsx (self, sheet = 'raw'):
        # create an ExcelWriter object
        writer = pd.ExcelWriter("data_join_" + self.loc + '.xlsx')

        # write the DataFrame to a sheet named "Sheet1"
        self.data_join.to_excel(writer, sheet_name = sheet, index=False)

        # save the Excel file
        writer.save()

In [88]:
Data = JoinData()

data_join = Data.data_join

C:\Users\OS\AppData\Local\Temp\ipykernel_14564\2952076249.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.cloud_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)
C:\Users\OS\AppData\Local\Temp\ipykernel_14564\2952076249.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.solar_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)
C:\Users\OS\AppData\Local\Temp\ipykernel_14564\2952076249.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.telopcode_clean = (raw_.append(table_add)).sort_values('target_date', ascending=True).reset_index(drop=True)


In [90]:
sheet = 'raw'
# create an ExcelWriter object
writer = pd.ExcelWriter("data_join_" + 'v1' + '.xlsx')

# write the DataFrame to a sheet named "Sheet1"
Data.data_join.to_excel(writer, sheet_name = sheet, index=False)

# save the Excel file
writer.save()
writer.close()

C:\Users\OS\AppData\Local\Temp\ipykernel_14564\3433137427.py:9: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
